In [6]:
import redshift_connector
import pandas as pd
import os

In [15]:
host = os.environ['host']
database = os.environ['database']
user = os.environ['user']
password = os.environ['password']
schema = os.environ['schema']

In [16]:
conn = redshift_connector.connect(
     host=host,
     database=database,
     user=user,
     password=password
  )

In [17]:
cursor = conn.cursor()

### 1.	Our client Barilla would like to know how many customers purchased their pasta products during February 2020. 

In [ ]:
query = f"""
SELECT COUNT(DISTINCT member_id) 
FROM {schema}.groceryorders
WHERE
brand = 'Barilla'
AND
order_date BETWEEN '2020-02-01' AND '2020-02-29'
"""

In [23]:
conn = redshift_connector.connect(
     host=host,
     database=database,
     user=user,
     password=password
  )
cursor = conn.cursor()
cursor.execute(query)
result = cursor.fetch_dataframe()
result

,count
0,603


### 2. We would like to present the monthly grocery sales for each vendor and provide the sales in dollars and also as percentages. In addition, the following vendors should be transformed as kroger: kingSoopers, smiths, fredMeyer, frys, picknsave & ralphs. The following vendor should be considered as albertsons: safeway. 

In [62]:
query = f"""
SELECT
CASE
    WHEN vendor = 'kingSoopers' THEN 'kroger'
    WHEN vendor = 'smiths' THEN 'kroger'
    WHEN vendor = 'fredMeyer' THEN 'kroger'
    WHEN vendor = 'frys' THEN 'kroger'
    WHEN vendor = 'picknsave' THEN 'kroger'
    WHEN vendor = 'ralphs' THEN 'kroger'
    WHEN vendor = 'safeway' THEN 'albertsons'
    ELSE vendor
END AS group_vendors,
to_char(order_date, 'MM') as month_order,
to_char(order_date, 'YYYY') as year_date,
SUM(cost) as total_cost,
SUM(cost) * 100 / SUM(SUM(cost)) OVER(PARTITION BY month_order,year_date ) AS percentage_cost
FROM {schema}.groceryorders
GROUP BY
group_vendors,
month_order,
year_date
"""

In [63]:
conn = redshift_connector.connect(
     host=host,
     database=database,
     user=user,
     password=password
  )
cursor = conn.cursor()

cursor.execute(query)
result = cursor.fetch_dataframe()
result

,group_vendors,month_order,year_date,total_cost,percentage_cost
0,walmartGrocery,01,2020,23527.78,84.360225
1,instacart,01,2020,10.39,0.037254
2,amazonFresh,01,2020,235.44,0.844184
3,shipt,01,2020,1009.74,3.620482
4,kroger,01,2020,3106.31,11.137855
...,...,...,...,...,...
75,instacart,12,2020,524.88,9.460485
76,kroger,12,2020,699.01,12.599020
77,heb,12,2020,127.06,2.290141
78,amazonFresh,12,2020,102.52,1.847830
